In [ ]:
import torch
import cv2
import numpy as np
from albumentations.pytorch import ToTensorV2
import albumentations as A
from torch.autograd import Variable
import os

from pytorch_dcsaunet.DCSAU_Net import Model

In [ ]:
def get_transform():
    return A.Compose([
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()  # ✅ 여기만 수정됨
    ])

In [3]:
def inference_single_image(image_path, model_path, output_path='result.png'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 이미지 불러오기
    original_image = cv2.imread(image_path)
    image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    transform = get_transform()
    transformed = transform(image=image)
    image_tensor = transformed['image'].unsqueeze(0).to(device)  # [1, C, H, W]

    # 모델 로딩
    # model = DCSAU_Net()  # 모델 클래스 초기화
    # model.load_state_dict(torch.load(model_path, map_location=device))
    # model.to(device)
    # model.eval()

    # 그대로 불러오기
    model = torch.load(model_path, map_location=device)
    model.to(device)
    model.eval()

    

    # 추론
    with torch.no_grad():
        output = model(image_tensor)
        output = torch.sigmoid(output)
        output = (output >= 0.5).float()

    # 결과 마스크 처리 및 저장
    mask = output.squeeze().cpu().numpy()  # [H, W]
    mask = (mask * 255).astype(np.uint8)

    # 저장
    cv2.imwrite(output_path, mask)
    print(f"Inference done! Result saved to: {output_path}")

In [4]:
image_path = 'output.png'  
model_path = 'assets/epoch_last.pth'  
output_path = 'output_mask.png'
inference_single_image(image_path, model_path, output_path)

Inference done! Result saved to: output_mask.png


/home/song/anaconda3/envs/dcs/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1702400441250/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


In [5]:
model = torch.load('assets/epoch_last.pth')